# Container forcast using raindom forest model

In this project, we aims the build a prediction model for container throughput in Thailand port. 

We first load all the data collected throughout year 2001-2021. This included inbound and outbound container throughput. As for the features, we use features as follows

- Consumer price index
- Export value
- Import value
- GDP constant
- Inflation rate
- Interest rate
- Manufacture product index
- Population
- Unemployment rate
- USD to THB conversion rate

In [3]:
import numpy as np
import pandas as pd

# Loading label for model prediction

In [4]:
label_df = pd.read_csv("container_throughput_label.csv")
label_df.head()


,year,month,inbound,outbound
0,2002,January,107493,96214
1,2002,February,97798,97257
2,2002,March,111474,112393
3,2002,April,101110,107746
4,2002,May,112976,119299


# Load features for model prediction

In [5]:
exportval_df = pd.read_csv("export_value.csv")
gdp_df = pd.read_csv("GDP_constant.csv")
importval_df = pd.read_csv("import_value.csv")
inflate_df = pd.read_csv("inflation_%.csv")
interest_df = pd.read_csv("interest_rate.csv")
manu_df = pd.read_csv("manufac_prod_index.csv")
pop_df = pd.read_csv("population.csv")
unemp_df = pd.read_csv("unemployment.csv")
ex_df = pd.read_csv("usd_thb.csv")
cons_df = pd.read_csv("consumer_price_index.csv")

In [6]:
exportval_df = exportval_df.drop('id', axis=1)

exportval_df = exportval_df[["year", "month", "export_value"]]
exportval_df['export_value'] = exportval_df['export_value'].map(lambda x: x.replace(',', ''))
exportval_df['export_value'] = pd.to_numeric(exportval_df['export_value'])

In [7]:
gdp_df = gdp_df.drop('id', axis=1)

gdp_df = gdp_df[["year", "month", "GDP_constant"]]
gdp_df['GDP_constant'] = gdp_df['GDP_constant'].map(lambda x: x.replace(',', ''))
gdp_df['GDP_constant'] = pd.to_numeric(gdp_df['GDP_constant'])

In [8]:
importval_df = importval_df.drop('id', axis=1)

importval_df = importval_df[["year", "month", "import_value"]]
importval_df['import_value'] = importval_df['import_value'].map(lambda x: x.replace(',', ''))
importval_df['import_value'] = pd.to_numeric(importval_df['import_value'])

In [9]:
inflate_df = inflate_df.drop('id', axis=1)

inflate_df = inflate_df[["year", "month", "inflation_percentage_change"]]
inflate_df['inflation_percentage_change'] = inflate_df['inflation_percentage_change'].map(lambda x: x.replace('%', ''))
inflate_df['inflation_percentage_change'] = pd.to_numeric(inflate_df['inflation_percentage_change'])

In [10]:
interest_df = interest_df.drop('id', axis=1)

interest_df = interest_df[["year", "month", "interest_rate"]]
interest_df['interest_rate'] = pd.to_numeric(interest_df['interest_rate'])

In [11]:
manu_df = manu_df.drop('id', axis=1)

manu_df = manu_df[["year", "month", "manufac_prod_index"]]
manu_df['manufac_prod_index'] = pd.to_numeric(manu_df['manufac_prod_index'])

In [12]:
pop_df = pop_df.drop('id', axis=1)

pop_df = pop_df[["year", "month", "population"]]
pop_df['population'] = pop_df['population'].map(lambda x: x.replace(',', ''))
pop_df['population'] = pd.to_numeric(pop_df['population'])

In [13]:
unemp_df = unemp_df.drop('id', axis=1)

unemp_df = unemp_df[["year", "month", "unemployment_rate"]]
unemp_df = unemp_df.replace(' n.a. ', np.nan)
unemp_df['unemployment_rate'] = pd.to_numeric(unemp_df['unemployment_rate'])

In [14]:
ex_df = ex_df.drop('id', axis=1)

ex_df = ex_df[["year", "month", "exchange_rate"]]
ex_df = ex_df.replace(' n.a. ', np.nan)
ex_df['exchange_rate'] = pd.to_numeric(ex_df['exchange_rate'])

In [15]:
cons_df = cons_df.drop('id', axis=1)

cons_df = cons_df[["year", "month", "consumer_price_index"]]
cons_df = cons_df.replace(' n.a. ', np.nan)
cons_df['consumer_price_index'] = pd.to_numeric(cons_df['consumer_price_index'])

# Merge all Dataframe into one

In [16]:
months = ['January', 'February', 'March', 'April', 'May', 
'June', 'July', 'August', 'September', 'October', 'November', 'December']

f_df = pd.DataFrame(data={'year': np.array([[year]*12 for year in range(2001,2022)]).flatten(),
                        'month': months*21 })


In [17]:
f_df = pd.merge(left=f_df, right=exportval_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=gdp_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=importval_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=inflate_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=interest_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=manu_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=pop_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=unemp_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=ex_df, how='left', on=['year','month'])
f_df = pd.merge(left=f_df, right=cons_df, how='left', on=['year','month'])

In [18]:
f_df.head()

,year,month,export_value,GDP_constant,import_value,inflation_percentage_change,interest_rate,manufac_prod_index,population,unemployment_rate,exchange_rate,consumer_price_index
0,2001,January,279973.0,459359.0,255061.0,0.72,2.5,52.47,62308887.0,5.73,43.12,68.8
1,2001,February,279973.0,459359.0,255061.0,0.44,2.0,53.02,62308887.0,4.25,42.64,69.1
2,2001,March,279973.0,459359.0,255061.0,0.00,2.0,52.18,62308887.0,4.04,43.90,69.1
3,2001,April,283056.0,442241.0,255379.0,0.72,2.0,51.29,62308887.0,4.06,45.46,69.6
4,2001,May,283056.0,442241.0,255379.0,0.29,2.0,52.51,62308887.0,4.24,45.48,69.8
